In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

import torch, torchvision
import torch.nn.functional as F
from torch import nn, optim
from torchvision import transforms, datasets
import os
import copy, os
from tqdm import trange

### 세팅 

In [2]:
# Computational device
# Device will be set to GPU if it is available.(you should install valid Pytorch version with CUDA. Otherwise, it will be computed using CPU)
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
DIR = './results/gridsearch16'
print("Using Device:", DEVICE)

Using Device: cuda


In [3]:
# Fashion MNIST dataset
trainset = datasets.FashionMNIST(
    root      = './.data/', train = True,
    download  = True,
    transform = transforms.ToTensor())
testset = datasets.FashionMNIST(
    root      = './.data/', train     = False,
    download  = True,
    transform = transforms.ToTensor())

In [4]:
SELECT_NORMAL = 2 # Set 2 class as train dataset.
trainset.data = trainset.data[trainset.targets == SELECT_NORMAL]
trainset.targets = trainset.targets[trainset.targets == SELECT_NORMAL] # Set 2 class as train dataset.

test_label = [2,4,6] # Define actual test class that we use
actual_testdata = torch.isin(testset.targets, torch.tensor(test_label))
testset.data = testset.data[actual_testdata]
testset.targets = testset.targets[actual_testdata]

test_loader = torch.utils.data.DataLoader(
    dataset     = testset, batch_size  = 1,
    shuffle     = False,num_workers = 2)

train_data_size = len(trainset)
test_data_size = len(testset)

print("Train data size:", train_data_size, "Test data size:", test_data_size)

Train data size: 6000 Test data size: 3000


#### 데이터 증강 기법 사용 class 

In [5]:
class GaussianNoise(nn.Module):
    def __init__(self, std=0.1):
        super().__init__()
        self.std = std

    def forward(self, x):
        if self.training:
            noise = x.data.new(x.size()).normal_(0, self.std)
            return x + noise
        return x

In [6]:
# 몇 배로 Augmentation을 할 것인지 알려주면 해당 배수만큼 Augmentation을 수행하는 클래스
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(15),                  # 20도는 좀 크고, 15도 이하 권장
    transforms.RandomCrop(28, padding=2),           # shift 효과
    GaussianNoise(0.1),                          # Gaussian Noise 추가  
])

class AugmentedDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transform=None, augmentation_factor=1):
        '''
        dataset: 원본 데이터셋
        transform: 증강을 위한 transform
        augmentation_factor: 몇 배로 Augmentation (1 포함)
        '''
        self.dataset = dataset
        self.transform = transform
        self.augmentation_factor = augmentation_factor
        self.original_length = len(dataset)

    def __len__(self):
        # 전체 데이터 수 = 원본 * 배수
        return self.original_length * self.augmentation_factor

    def __getitem__(self, idx):
        # 원본 인덱스
        original_idx = idx % self.original_length
        x, y = self.dataset[original_idx]

        # factor == 1 이거나 첫 번째 패스는 원본 사용
        if self.augmentation_factor > 1 and idx >= self.original_length:
            if self.transform:
                x = self.transform(x)
        
        return x, y


In [7]:
# 데이터셋을 먼저 train과 val로 나누고, train에 대해서만 증강을 적용
n_val = int(len(trainset) * 0.2)
n_train = len(trainset) - n_val
BATCH_SIZE = 1024

augset, valset = torch.utils.data.random_split(trainset, [n_train, n_val], generator=torch.Generator().manual_seed(2025))

augset = AugmentedDataset(augset, transform=transform, augmentation_factor=5) # augmentation_factor = 5 for baseline 모델 찾기 
# valset은 증강을 적용하지 않음

train_loader = torch.utils.data.DataLoader(
    dataset     = augset, batch_size  = BATCH_SIZE,
    shuffle     = True,num_workers = 0) 

val_loader = torch.utils.data.DataLoader(
    dataset     = valset, batch_size = BATCH_SIZE,
    shuffle     = False,num_workers = 0)

# data size check
print("Train data size:", len(augset),"Val data size:", len(valset),"Test data size:", len(testset))

Train data size: 24000 Val data size: 1200 Test data size: 3000


### 모델 및 Training Setting 

In [8]:
class EarlyStopping():
    def __init__(self, patience=10, verbose=False, delta=0):
        '''
        patience (int): 얼마나 기다릴지
        verbose (bool): True일 경우 각 epoch의 loss 출력
        delta (float): 개선이 되었다고 인정되는 최소한의 loss
        '''
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.inf
        self.delta = delta

    def __call__(self, val_loss, model):
        score = -val_loss # validation loss가 작을수록 좋다고 가정

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

### 모델 및 loss 불러오기 

In [9]:
import torch.nn as nn
import model

def get_model_classes():
    """
    model 폴더 내에서 nn.Module 기반 클래스만 자동으로 dict로 반환 (instance가 아니라 class 반환)
    """
    model_classes = {}
    for k in dir(model):
        obj = getattr(model, k)
        if isinstance(obj, type) and issubclass(obj, nn.Module) and obj.__module__.startswith('model.'):
            model_classes[k] = obj  # <-- instance() 하지 않고 class 자체만 저장
    return model_classes

# 이제 model_classes는 {name: class} 형태로만 보관
model_classes = get_model_classes()
print("Available models:", model_classes.keys())

Available models: dict_keys(['Autoencoder', 'CAE', 'CVAE', 'DeepCAE', 'DeepCVAE', 'DeepVAE', 'DenoisingAutoencoder', 'DiffusionUNet', 'GANomaly', 'HybridCAE', 'RobustAutoencoder', 'SimpleDDPM', 'SkipConnectionAutoencoder', 'TransformerAnomalyDetector', 'VAE'])


In [10]:
# loss function
# loss function 추천 조합
from loss.losses import FlexibleLoss, FlexibleDiffusionLoss

reconstruction_loss = {
    "MSE": FlexibleLoss(mode="mse"),
    "MSE+Gradient": FlexibleLoss(mode="mse+gradient", beta=1.0, gamma=0.1),
    "MSE+MS-SSIM": FlexibleLoss(mode="mse+ms-ssim", beta=1.0, alpha=0.3),
    "Charbonnier+MS-SSIM": FlexibleLoss(mode="charbonnier+ms-ssim", beta=1.0, alpha=0.5),
    "Charbonnier+Gradient": FlexibleLoss(mode="charbonnier+gradient", beta=1.0, gamma=0.1),
}

diffusion_loss = {
    "MSE": FlexibleDiffusionLoss(mode="mse"),
    "MSE+Gradient": FlexibleDiffusionLoss(mode="mse+gradient", beta=1.0, alpha=0.1),
    "Charbonnier": FlexibleDiffusionLoss(mode="charbonnier", beta=1.0),
    "Charbonnier+Gradient": FlexibleDiffusionLoss(mode="charbonnier+gradient", beta=1.0, alpha=0.1),
    "Charbonnier+Gradient+Tv": FlexibleDiffusionLoss(mode="charbonnier+gradient+tv", beta=1.0, alpha=0.1, gamma=0.1),
}
loss_functions = {
    "reconstruction": reconstruction_loss,
    "diffusion": diffusion_loss,
}

print("Available reconstruction loss functions:", reconstruction_loss.keys())
print("Available diffusion loss functions:", diffusion_loss.keys())

Available reconstruction loss functions: dict_keys(['MSE', 'MSE+Gradient', 'MSE+MS-SSIM', 'Charbonnier+MS-SSIM', 'Charbonnier+Gradient'])
Available diffusion loss functions: dict_keys(['MSE', 'MSE+Gradient', 'Charbonnier', 'Charbonnier+Gradient', 'Charbonnier+Gradient+Tv'])


In [11]:
def model_delete(models, loss_functions, checkpoint_dir):
    '''
    이미 학습된 모델 + loss 조합을 models, loss_functions에서 제거
    checkpoint_dir/{model_name}_{loss_name}.pth 존재 여부로 판단
    '''
    models_to_delete = []
    models_not_to_delete = []
    for model_name in list(models.keys()):
        # 현재 모델이 사용 가능한 loss 목록
        if hasattr(models[model_name], 'T'):
            losses = list(loss_functions['diffusion'].keys())
        else:
            losses = list(loss_functions['reconstruction'].keys())
        
        # 해당 모델의 모든 loss 조합이 학습되었는지 확인
        all_ckpt_exist = all(
            os.path.exists(os.path.join(checkpoint_dir, f"{model_name}_{loss.replace('+','and')}.pth"))
            for loss in losses
        )

        if all_ckpt_exist:
            models_to_delete.append(model_name)
        else:
            models_not_to_delete.append(model_name)
    
    print(f"✅ {len(models_to_delete)} models and loss functions will be deleted: {models_to_delete}")
    print(f"✅ {len(models_not_to_delete)} models and loss functions will be kept: {models_not_to_delete}")
    # 실제 삭제
    for model_name in models_to_delete:
        del models[model_name]

    return models
# 모델과 loss function 조합을 삭제
model_classes = model_delete(model_classes, loss_functions, DIR)
print("Remaining models:", model_classes.keys())

✅ 0 models and loss functions will be deleted: []
✅ 15 models and loss functions will be kept: ['Autoencoder', 'CAE', 'CVAE', 'DeepCAE', 'DeepCVAE', 'DeepVAE', 'DenoisingAutoencoder', 'DiffusionUNet', 'GANomaly', 'HybridCAE', 'RobustAutoencoder', 'SimpleDDPM', 'SkipConnectionAutoencoder', 'TransformerAnomalyDetector', 'VAE']
Remaining models: dict_keys(['Autoencoder', 'CAE', 'CVAE', 'DeepCAE', 'DeepCVAE', 'DeepVAE', 'DenoisingAutoencoder', 'DiffusionUNet', 'GANomaly', 'HybridCAE', 'RobustAutoencoder', 'SimpleDDPM', 'SkipConnectionAutoencoder', 'TransformerAnomalyDetector', 'VAE'])


### Trainer 실행 

In [12]:
# Check the shape of a batch from train_loader and test_loader
train_images, train_labels = next(iter(train_loader))
test_images, test_labels = next(iter(test_loader))

print("Train batch image shape:", train_images.shape)
print("Train batch label shape:", train_labels.shape)
print("Test batch image shape:", test_images.shape)
print("Test batch label shape:", test_labels.shape)

Train batch image shape: torch.Size([1024, 1, 28, 28])
Train batch label shape: torch.Size([1024])
Test batch image shape: torch.Size([1, 1, 28, 28])
Test batch label shape: torch.Size([1])


In [13]:
def dir_clear(log_dir):
    '''
    log_dir: 로그를 저장할 디렉토리 경로
    하위 폴더 및 파일을 삭제
    '''
    if os.path.exists(log_dir):
        for root, dirs, files in os.walk(log_dir, topdown=False):
            for name in files:
                os.remove(os.path.join(root, name))
            for name in dirs:
                os.rmdir(os.path.join(root, name))
        print(f"✅ {log_dir} cleared.")
    else:
        print(f"❌ {log_dir} does not exist. No need to clear.")

dir_clear(DIR + "/logs")
dir_clear(DIR + "/checkpoints")
dir_clear(DIR + "/eval_results")

✅ ./results/gridsearch16/logs cleared.
✅ ./results/gridsearch16/checkpoints cleared.
✅ ./results/gridsearch16/eval_results cleared.


In [ ]:
EPOCHS = 100
PATIENCE = 20
# GridSearchTrainerfp16
from trainer import GridSearchTrainerFP16
trainer = GridSearchTrainerFP16(
    models=model_classes,
    criterions_dict=loss_functions,
    train_loader=train_loader,
    val_loader=val_loader,
    n_epochs=EPOCHS,
    patience=PATIENCE,
    save_dir=f'{DIR}/checkpoints',
    verbose=False, 
    device=DEVICE,
    lr=1e-3 * BATCH_SIZE / 256, # default learning rate for AdamW
    log_dir=f'{DIR}/logs',
)
results = trainer.run()

results_df = pd.DataFrame(results)
# Save the results to a CSV file
results_df.to_csv(f'{DIR}/training.csv', index=False)

Total Models: 15
Reconstruction Losses: 5 Diffusion Losses: 5
Total Combinations: 75
▶ Training [Autoencoder] with [MSE] (FP16)


Autoencoder | MSE (FP16):   1%|          | 1/100 [00:07<11:55,  7.23s/it]

In [ ]:
results_df

,model,loss,best_val_loss,best_epoch,gpu_peak_usage(MB),save_path
0,Autoencoder,MSE,0.047587,2,29.301270,./results/gridsearch16/checkpoints/Autoencoder...
1,Autoencoder,MSE+Gradient,0.063891,2,29.301758,./results/gridsearch16/checkpoints/Autoencoder...
2,CAE,MSE,0.013513,2,45.984375,./results/gridsearch16/checkpoints/CAE_MSEfp16...
3,CAE,MSE+Gradient,0.026053,2,26.442871,./results/gridsearch16/checkpoints/CAE_MSEandG...
4,CVAE,MSE,0.001912,2,38.269043,./results/gridsearch16/checkpoints/CVAE_MSEfp1...
5,CVAE,MSE+Gradient,0.000426,2,38.269043,./results/gridsearch16/checkpoints/CVAE_MSEand...
6,DeepCAE,MSE,0.029551,2,34.267578,./results/gridsearch16/checkpoints/DeepCAE_MSE...
7,DeepCAE,MSE+Gradient,0.063687,2,34.267578,./results/gridsearch16/checkpoints/DeepCAE_MSE...
8,DeepCVAE,MSE,0.000274,2,49.350586,./results/gridsearch16/checkpoints/DeepCVAE_MS...
9,DeepCVAE,MSE+Gradient,0.000313,2,49.350586,./results/gridsearch16/checkpoints/DeepCVAE_MS...


In [ ]:
print(f"tensorboard --logdir {DIR}/logs") # tensorboard --logdir ./results/gridsearch16/logs/1

tensorboard --logdir ./results/gridsearch16/logs


### eval

In [ ]:
import os
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, f1_score, accuracy_score
from tqdm import tqdm

class Evaluator:
    def __init__(self, checkpoint_dir, val_loader, test_loader, device=None, percentile=0.95, save_dir='./eval_results'):
        self.checkpoint_dir = checkpoint_dir
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device if device else torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.percentile = percentile
        self.save_dir = save_dir
        os.makedirs(self.save_dir, exist_ok=True)

    def load_model(self, model_name):
        """모델 이름으로부터 instance 생성"""
        model_class = get_model_classes()[model_name]
        model = model_class().to(self.device)
        return model

    def compute_score(self, model, x):
        """Diffusion → noise mse / AE, VAE, etc → recon mse"""
        if hasattr(model, 'T'):
            t = torch.randint(0, model.T, (x.size(0),), device=x.device)
            noise_pred, noise = model(x, t)
            return F.mse_loss(noise_pred, noise, reduction='none').view(x.size(0), -1).mean(dim=1)
        else:
            output = model(x)
            if isinstance(output, tuple):
                output = output[0]
            return F.mse_loss(output, x, reduction='none').view(x.size(0), -1).mean(dim=1)

    def get_scores(self, loader, model):
        """Validation 또는 Test set에 대한 score 계산"""
        model.eval()
        scores, labels = [], []
        with torch.no_grad():
            for x, y in tqdm(loader, desc="Scoring"):
                x = x.to(self.device)
                score = self.compute_score(model, x)
                scores.append(score.cpu())
                labels.append(y.cpu())
        return torch.cat(scores).numpy(), torch.cat(labels).numpy()

    def run(self):
        checkpoint_files = [f for f in os.listdir(self.checkpoint_dir) if f.endswith(".pth")]
        results = []

        for ckpt in checkpoint_files:
            print(f"\n▶ Evaluating {ckpt}")

            # 안전하게 loss_name에 _가 여러 개 들어가도 마지막만 split
            model_name, loss_name = ckpt[:-4].rsplit("_", 1)

            model = self.load_model(model_name)
            model.load_state_dict(torch.load(os.path.join(self.checkpoint_dir, ckpt)))

            # --------------------------
            # Threshold 계산 (Validation)
            # --------------------------
            val_scores, _ = self.get_scores(self.val_loader, model)
            threshold = np.percentile(val_scores, self.percentile * 100)
            print(f" >> Threshold ({self.percentile*100:.0f}%) = {threshold:.4f}")

            # --------------------------
            # Test 평가
            # --------------------------
            test_scores, test_labels = self.get_scores(self.test_loader, model)
            test_labels = (test_labels != 2).astype(int)  # class 2 = normal
            preds = (test_scores > threshold).astype(int)

            auc_score = roc_auc_score(test_labels, test_scores)
            precision, recall, _ = precision_recall_curve(test_labels, test_scores)
            pr_auc = auc(recall, precision)
            f1 = f1_score(test_labels, preds)
            acc = accuracy_score(test_labels, preds)

            print(f"ROC-AUC: {auc_score:.4f} | PR-AUC: {pr_auc:.4f} | F1: {f1:.4f} | ACC: {acc:.4f}")

            results.append({
                "checkpoint": ckpt,
                "threshold": threshold,
                "roc_auc": auc_score,
                "pr_auc": pr_auc,
                "f1": f1,
                "acc": acc
            })

        df = pd.DataFrame(results)
        return df


In [ ]:
evaluator = Evaluator(
    checkpoint_dir=f'{DIR}/checkpoints',
    val_loader=val_loader,
    test_loader=test_loader,
    device=DEVICE,
    percentile=0.95,
    save_dir=f'{DIR}/eval_results'  # ⭕ 저장할 디렉토리
)
eval_results = evaluator.run()
eval_results.to_csv(f'{DIR}/eval_results/eval_results.csv', index=False)  # ⭕ CSV로 저장


▶ Evaluating Autoencoder_MSEandGradientfp16.pth


Scoring: 100%|██████████| 2/2 [00:00<00:00, 46.51it/s]


 >> Threshold (95%) = 0.0828


Scoring: 100%|██████████| 3000/3000 [00:05<00:00, 510.26it/s] 


ROC-AUC: 0.5594 | PR-AUC: 0.6998 | F1: 0.1241 | ACC: 0.3603

▶ Evaluating Autoencoder_MSEfp16.pth


...

ROC-AUC: 0.5215 | PR-AUC: 0.6697 | F1: 0.0573 | ACC: 0.3420

▶ Evaluating VAE_MSEfp16.pth


Scoring: 100%|██████████| 2/2 [00:00<00:00, 41.67it/s]


 >> Threshold (95%) = 0.1235


Scoring: 100%|██████████| 3000/3000 [00:04<00:00, 631.00it/s] 

ROC-AUC: 0.5124 | PR-AUC: 0.6679 | F1: 0.0601 | ACC: 0.3437


In [ ]:
eval_results

,checkpoint,threshold,roc_auc,pr_auc,f1,acc
0,Autoencoder_MSEandGradientfp16.pth,0.082814,0.559424,0.699810,0.124144,0.360333
1,Autoencoder_MSEfp16.pth,0.083250,0.543121,0.691362,0.120769,0.359333
2,CAE_MSEandGradientfp16.pth,0.019127,0.400745,0.618889,0.115665,0.352667
3,CAE_MSEfp16.pth,0.019510,0.401724,0.616580,0.115665,0.352667
4,CVAE_MSEandGradientfp16.pth,0.122798,0.521094,0.664488,0.054337,0.338667
5,CVAE_MSEfp16.pth,0.125276,0.518763,0.663091,0.046980,0.337333
6,DeepCAE_MSEandGradientfp16.pth,0.078911,0.518472,0.655431,0.063089,0.336667
7,DeepCAE_MSEfp16.pth,0.054190,0.426635,0.612067,0.048479,0.332667
8,DeepCVAE_MSEandGradientfp16.pth,0.121536,0.520331,0.663893,0.040462,0.336000
9,DeepCVAE_MSEfp16.pth,0.119235,0.522489,0.666580,0.051650,0.339000
